# Creating Commitment Transactions with HTLC in Transit

In this section, we'll create the Commitment Transactions with HTLC in transit from scratch using Python. We'll break down each part of the transaction, explain how it's constructed and signed, and detail the messages exchanged between peers to share the necessary information. Finally, we'll test everything using Bitcoin Core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network BOLT #3: '[Commitment Transactions](https://github.com/lightning/bolts/blob/master/03-transactions.md#commitment-transaction)'
- Finite Fields, Elliptic Curves and Serialization e.g. [Programming Bitcoin](https://github.com/jimmysong/programmingbitcoin), in particular [Chapter 1](https://github.com/jimmysong/programmingbitcoin/blob/master/ch01.asciidoc), [Chapter2](https://github.com/jimmysong/programmingbitcoin/blob/master/ch02.asciidoc), [Chapter 3](https://github.com/jimmysong/programmingbitcoin/blob/master/ch03.asciidoc) and [Chapter 4](https://github.com/jimmysong/programmingbitcoin/blob/master/ch04.asciidoc).

## HTLC (Hashed Time Locked Contracts)

Once the channel was stablished, the channel can be used to make payments via HTLCs (Hashed Time Locked Contracts).

Changes are sent in batches: one or more update_ messages are sent before a commitment_signed message, as in the following diagram:

    +-----------+                               +-----------+
    |           |--(1)---- update_add_htlc ---->|           |
    |           |--(2)---- update_add_htlc ---->|           |
    |           |<-(3)---- update_add_htlc -----|           |
    |           |                               |           |
    |           |--(4)--- commitment_signed --->|           |
    |   Alice   |<-(5)---- revoke_and_ack ------|    Bob    |
    |           |                               |           |
    |           |<-(6)--- commitment_signed ----|           |
    |           |--(7)---- revoke_and_ack ----->|           |
    |           |                               |           |
    |           |--(8)--- commitment_signed --->|           |
    |           |<-(9)---- revoke_and_ack ------|           |
    +-----------+                               +-----------+





## Setup
### Requirements

For this exercise we'll run the previous notebook to create the channel funding transaction beetween Alice and Bob.

**You'll need to edit these next line for your local setup.**